In [1]:
## Load all libraries


import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier


In [16]:
positive_words = [line.strip() for line in open('data/positive-words.txt', encoding='latin-1').readlines() if line.strip() and not line.startswith(';')]
negative_words = [line.strip() for line in open('data/negative-words.txt', encoding='latin-1').readlines() if line.strip() and not line.startswith(';')]

# Please download the word embeddings from https://nlp.stanford.edu/projects/glove/
# This file here is a symlink only

def get_embeddings():
    with open('data/glove.840B.300d.txt') as fp:
        labels = []
        rows = []
        for i, line in enumerate(fp):
            items = line.rstrip().split(' ')
            if len(items) == 2:
                continue
            labels.append(items[0])
            rows.append(np.array([float(x) for x in items[1:]], 'f'))
        arr = np.vstack(rows)
        print(arr.shape)
        return pd.DataFrame(arr, index=labels, dtype='f')


In [17]:
word_vectors = get_embeddings()

(2196017, 300)


In [ ]:
word_vectors[:3]

In [8]:
negative_words[:10]

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted']

In [19]:
positive_vectors = word_vectors.loc[positive_vectors.index.intersection(positive_words)].dropna()
negative_vectors = word_vectors.loc[negative_vectors.index.intersection(negative_words)].dropna()

In [110]:
data = pd.concat([positive_vectors, negative_vectors])
targets = ([1 for word in positive_vectors.index] + [-1 for word in negative_vectors.index])
labels = data.index
train_data, test_data, train_target, test_target, train_labels, test_labels = train_test_split(data, targets, labels, test_size=0.1)

In [111]:
test_data[:10]

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
talents,0.079738,0.225430,0.481470,-0.215010,-0.242330,0.242570,0.253780,0.295400,-0.611930,2.487300,...,-0.511370,0.061727,-0.230910,0.267960,-0.066753,0.127250,0.288560,-0.149570,-0.231650,-0.091760
compliment,-0.482480,0.000423,-0.266020,-0.285650,-0.059128,0.205160,0.142190,-0.319580,-0.545840,1.268800,...,0.218690,0.212340,-0.302480,0.171490,0.343380,-0.001801,0.173300,-0.157050,-0.097721,0.265140
authoritative,-0.363900,-0.260220,0.070223,0.621990,0.495600,0.454250,0.376440,0.265910,0.406690,1.210200,...,-0.342780,-0.336240,-0.031502,0.217860,0.299480,-0.244840,0.438770,-0.374580,0.187750,-0.283670
providence,0.042244,-0.433010,0.166920,-0.593430,0.109380,0.113960,-0.030350,0.717000,0.418350,1.349000,...,0.173810,-0.027388,-0.387370,0.473360,-0.013823,-0.022718,0.668730,-0.116850,-0.290870,-0.217070
well-mannered,-0.530950,0.145910,-0.319220,-0.179750,0.314400,0.354560,0.048244,-0.335420,0.159160,0.084073,...,-0.406210,-0.358610,0.113150,0.566350,0.690270,0.448130,0.282260,-0.314580,-0.121280,0.069098
dismaying,0.263180,0.399460,0.919330,0.126840,-0.102450,0.019753,-0.160810,0.511620,-0.128500,-0.128760,...,-0.029655,0.152860,0.253370,0.086322,0.285280,0.090775,-0.109520,-0.015017,-0.500690,-0.366520
inefficiency,-1.105700,0.461170,0.839480,0.015746,-0.914220,0.027020,0.132380,-0.080266,-0.488800,0.949530,...,0.231600,0.177470,0.726290,-0.057088,-0.502810,0.448690,0.281280,0.564260,0.041583,0.049635
concession,0.344790,-0.487610,0.999400,-0.004551,0.336730,0.258800,-0.007839,-0.066173,0.037484,1.762600,...,-0.420230,0.494760,0.236870,-0.029079,0.439590,0.019844,0.312140,-0.166460,0.319590,0.213910
anti-social,-0.203020,0.431070,-0.496370,0.275530,-0.277230,-0.365060,0.343200,-0.127080,-0.101390,0.975830,...,-0.378850,0.237820,0.488760,0.641170,0.410270,0.692650,0.094197,0.366780,-0.074061,0.064191
splendid,0.237170,0.025011,-0.031719,-0.274420,-0.276110,0.355150,0.427570,0.124220,-0.528210,1.707400,...,-0.341590,0.006202,-0.372650,0.253760,-0.003798,0.334760,0.513250,0.141040,-0.027448,0.044597


In [127]:
model = SGDClassifier(max_iter=200, loss='log')
model.fit(train_data, train_target)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=200, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [128]:
accuracy_score(model.predict(test_data), test_target)

0.9622356495468278

In [164]:
import re
TOKEN_RE = re.compile(r"\w.*?\b")

def get_sentiment(input_str):
    embeddings = [word_vectors.loc[token.casefold()] for token in TOKEN_RE.findall(input_str)]
    predictions = model.predict_log_proba(embeddings)
    return (predictions[:, 1] - predictions[:, 0]).mean()


In [173]:
#Testing on trivial words
print(get_sentiment("good"))
print(get_sentiment("bad"))
print(get_sentiment("ugly"))
print(get_sentiment("The good, the bad and the ugly"))

8.233151308586075
-9.688738095325608
-16.99371687959816
-1.9686040217022105


In [174]:
print(get_sentiment("My name is Emily"))
print(get_sentiment("My name is Yvonne"))
print(get_sentiment("My name is Shaniqua"))


0.4445063989124153
-0.4437736356852131
-2.4018470012600437


In [175]:
print(get_sentiment("My name is James"))
print(get_sentiment("My name is Mario"))
print(get_sentiment("My name is DeShawn"))


0.2332782816924801
0.08885950109471236
-0.4140725296787715
